In [ ]:
# Data analysis packages:
import pandas as pd
import numpy as np
# pd.set_option('display.float_format', lambda x: '%.5f'%x)
#from datetime import datetime as dt

# Visualization packages:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Import all necessary libraries.

import warnings
# import plots as p
import function as f
# import data_testing as dt
# import data_prep as dp

import itertools
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.gofplots import qqplot
import statsmodels.api as sm
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import datetime as dt
plt.style.use('seaborn-darkgrid')
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

Energy savings has two keys elements:
- forecasting energy consumption without any energy saving measures 
- forecasting energy consumption after certain energy saving measures

This notebook predict 

# Dataset:

In [ ]:
# training data
train = pd.read_csv("data/train.csv", parse_dates=['timestamp'])
weather_train = pd.read_csv("data/weather_train.csv", parse_dates=['timestamp'])
metadata = pd.read_csv("data/building_metadata.csv")

print('Size of training data', train.shape)
print('Mem. size of original training data', train.memory_usage().sum()/1024**2)
print('----------------------------------')
print('Size of training weather data', weather_train.shape)
print('Mem. size of original training weather data', weather_train.memory_usage().sum()/1024**2)
print('----------------------------------')
print('Size of building meta data', metadata.shape)
print('Mem. size of original building meta data', metadata.memory_usage().sum()/1024**2)

# df.memory_usage().sum() / 1024**2 

## Reducing memory

In [ ]:
## Reducing memory
print('Mem. size of reduced training data', train_df.shape)
train_df = f.reduce_mem_usage(train)
print('----------------------------------')
print('Mem. size of reduced training weather data', weather_train_df.shape)
weather_train_df = f.reduce_mem_usage(weather_train)
print('----------------------------------')
print('Mem. size of reduced building meta data', building_meta_df.shape)
building_meta_df = f.reduce_mem_usage(metadata)

***there are 15 different site_ids***

## Training data

In [ ]:
print(train_df.shape)
train_df.head(3)

## Metadata

In [ ]:
print(building_meta_df.shape)
building_meta_df.head(3)

## Weather data

In [ ]:
print(weather_train_df.shape)
weather_train_df.head(3)

# Checking missing values 

In [ ]:
train_df.isna().sum()/len(train_df)

In [ ]:
building_meta_df.isna().sum()#/len(building_meta_df)

In [ ]:
weather_train_df.isna().sum()#/len(weather_train_df)

# Replacing meter's

In [ ]:
print(train_df['meter'].value_counts())

In [ ]:
train_df['meter'].replace({0:"Electricity",1:"ChilledWater", 2:"Steam",3:"HotWater"}, inplace=True)
train_df['meter'].value_counts()

In [ ]:
import seaborn as sns 
sns.set(style='darkgrid')
# sns.set_style("darkgrid", {"axes.facecolor": ".9"})
# sns.set_context('talk')
sns.countplot(train_df['meter'], order=train_df['meter'].value_counts().index)
plt.title('Distribution of Meter Id Code')
plt.xlabel('Meter Id Code')
plt.ylabel('Frequency');

In [ ]:
print ("There are {} unique Buildings in the training data".format(train_df['building_id'].nunique()))

Like it is mentioned in the competition description, each building may or may not have all 4 meter

In [ ]:
train_df.groupby('meter')['meter_reading'].agg(['min','max','mean','median', 'count', 'std'])

## Building Metadata:

In [ ]:
# metadata
sns.countplot(building_meta_df['primary_use'], order=building_meta_df['primary_use'].value_counts().index)
plt.title('Distribution of Primary use of Buildings')
plt.xlabel('Building types')
plt.ylabel('Frequency');

In [ ]:
# Building floor count
sns.countplot(building_meta_df['floor_count'], order=building_meta_df['floor_count'].value_counts().index)
plt.title('Floor counts')
plt.xlabel('Floors')
plt.ylabel('Frequency');

In [ ]:
# year built 
sns.countplot(building_meta_df['year_built'], order=building_meta_df['year_built'].value_counts().index)
plt.title('Year built')
plt.xlabel('Years')
plt.ylabel('Frequency');

In [ ]:
# fig = px.scatter(metadata, y="square_feet", x="year_built", color="primary_use", size="square_feet")
# fig.update_layout(showlegend=True)
# fig.show()

In [ ]:
for df in [train_df]:
    df['month'] = df['timestamp'].dt.month.astype("uint8")
    df['day_of_month'] = df['timestamp'].dt.day.astype("uint8")
    df['day_of_week'] = df['timestamp'].dt.dayofweek.astype("uint8")
    df['hour'] = df['timestamp'].dt.hour.astype("uint8")

In [ ]:
def split_timestamp(df):
    df['month'] = df['timestamp'].dt.month.astype('uint8')
    df['day_of_month'] = df['timestamp'].dt.day.astype('uint8')
    df['day_of_week'] = df['timestamp'].dt.dayofweek.astype('uint8')
    df['hour'] = df['timestamp'].dt.hour.astype('uint8')
    
    return df

### Grouping by meter and month

In [ ]:
train_df = split_timestamp(train_df)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
train_df.groupby(['meter', 'month'])['meter_reading'].agg(['max', 'mean', 'median', 'count', 'std'])

In [ ]:
train.drop(columns=['Month', 'DayOfMonth', 'DayOfWeek', 'Hour'], inplace=True)
train.head(1)

In [ ]:
train_df.groupby(['meter', 'day_of_week'])['meter_reading'].agg(['max', 'mean', 'median', 'count', 'std'])

In [ ]:
sns.distplot(np.log1p(train_df['meter_reading']), kde=False)
plt.title('Distribution of log of meter reading variable')

In [ ]:
sns.boxplot(train_df[train_df['meter'] == "Electricity"]['meter_reading']);
plt.title("Boxplot of Meter Reading Variable for the Meter Type: Electricity");
# We can see a few outliers here. 

In [ ]:
sns.boxplot(train_df[train_df['meter'] == "ChilledWater"]['meter_reading']);
plt.title("Boxplot of Meter Reading Variable for the Meter Type: ChilledWater");
# Not many outliers here.

In [ ]:
sns.boxplot(train_df[train_df['meter'] == "HotWater"]['meter_reading']);
plt.title("Boxplot of Meter Reading Variable for the Meter Type: HotWater");
# We can see a single value that is way off from the rest. 

In [ ]:
sns.boxplot(train_df[train_df['meter'] == "Steam"]['meter_reading']);
plt.title("Boxplot of Meter Reading Variable for the Meter Type: Steam");
# We can see a few outliers here. 

### Converting the dependent variable to logarithmic scale

In [ ]:
train_df.head()

In [ ]:
train_df['meter_reading'] = np.log1p(train_df['meter_reading'])
train_df.head()

In [ ]:
sns.distplot(train_df[train_df['meter'] == "Electricity"]['meter_reading'],kde=False)
plt.title("Distribution of Meter Reading per MeterID code: Electricity")

In [ ]:
sns.distplot(train_df[train_df['meter'] == "ChilledWater"]['meter_reading'],kde=False)
plt.title("Distribution of Meter Reading per MeterID code: Chilledwater")

In [ ]:
sns.distplot(train_df[train_df['meter'] == "Steam"]['meter_reading'],kde=False)
plt.title("Distribution of Meter Reading per MeterID code: Steam")

In [ ]:
sns.distplot(train_df[train_df['meter'] == "HotWater"]['meter_reading'],kde=False)
plt.title("Distribution of Meter Reading per MeterID code: Hotwater")

In [ ]:
train_df.head(3)

## Metadata 

In [ ]:
building_meta_df.info()
# Missing values in year_built and floor_count variables

In [ ]:
building_meta_df.head(3)

In [ ]:
cols = ['site_id','primary_use','year_built','floor_count','building_id']
for col in cols:
    print ("Number of Unique Values in the {} column are:".format(col),building_meta_df[col].nunique())

In [ ]:
cols = ['site_id','primary_use','floor_count']
for col in cols:
    print ("Unique Values in the {} column are:".format(col),building_meta_df[col].unique())
    print ("\n")

In [ ]:
sns.countplot(building_meta_df['site_id'])
plt.title("Count of Site_id in the Metadata table")
plt.xlabel("Site_Id")
plt.ylabel("Count")

In [ ]:
plt.figure(figsize=(8,6))
building_meta_df['primary_use'].value_counts().sort_values().plot(kind='bar')
plt.title("Count of Primary_Use Variable in the Metadata table")
plt.xlabel("Primary Use")
plt.ylabel("Count")
plt.xticks(rotation=90);
# Education, Office, Entertainment/Public Assembly, Public Services, Lodging/Residential form the bulk of Primary Use

In [ ]:
building_meta_df['square_feet'].describe()

In [ ]:
sns.boxplot(building_meta_df['square_feet']);

In [ ]:
building_meta_df['square_feet'] = np.log1p(building_meta_df['square_feet'])

In [ ]:
sns.distplot(building_meta_df['square_feet'])
plt.title("Distribution of Square Feet variable of Metadata Table")
plt.xlabel("Area in Square Feet")
plt.ylabel("Frequency");
# Looks like a normal distribution distribution

In [ ]:
sns.boxplot(building_meta_df['square_feet'])
plt.title("Box Plot of Square Feet Variable");
# There are a few outliers visible

In [ ]:
building_meta_df.groupby('primary_use')['square_feet'].agg(['mean','median','count']).sort_values(by='count')
# Parking has the highest average are although the count is less.
# Education has the highest count as can be seen in the countplot above.

In [ ]:
building_meta_df['year_built'].value_counts().sort_values().plot(kind='bar',figsize=(15,6))
plt.xlabel("Year Built")
plt.ylabel("Count")
plt.title("Distribution of Year Built Variable");

In [ ]:
building_meta_df['floor_count'].value_counts(dropna=False).sort_index().plot(kind='bar',figsize=(8,6))
plt.xlabel("Number of Floors")
plt.ylabel("Count of Buildings");
# Lot of missing values here as well
# Maximum number of floors is 26

In [ ]:
building_meta_df.groupby('floor_count')['square_feet'].agg(['count','mean','median']).sort_values(by='count')

In [ ]:
building_meta_df.groupby('primary_use')['square_feet'].agg(['count','mean','median']).sort_values(by='count')

In [ ]:
metadata['year_built'].fillna(-999, inplace=True)
metadata['year_built'] = metadata['year_built'].astype('int16')
metadata['floor_count'].fillna(-999, inplace=True)
metadata['floor_count'] = metadata['floor_count'].astype('float32')

In [ ]:
metadata.head(3)

## Weather Data (Train)

In [ ]:
weather_train_df.head(3)

In [ ]:
cols = ['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed']
for col in cols:
    print (" Minimum Value of {} column is {}".format(col,weather_train[col].min()))
    print (" Maximum Value of {} column is {}".format(col,weather_train[col].max()))
    print ("----------------------------------------------------------------------")

In [ ]:
weather_train_df.isna().sum()/len(weather_train_df)

In [ ]:
weather_train_df[['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_speed']].describe()

In [ ]:
print(weather_train['timestamp'].min())
print(weather_train['timestamp'].max())
print(len( weather_train['timestamp']))
# This data is from 1st Jan to 31st Dec 2016, similar to the timestamp of the training data

In [ ]:
cols = ['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_speed']
for ind,col in enumerate(weather_train[cols]):
    plt.figure(ind)
    sns.distplot(weather_train[col].dropna())

In [ ]:
cols = ['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_speed']
for ind,col in enumerate(weather_train[cols]):
    plt.figure(ind)
    sns.boxplot(weather_train[col].dropna())

# Merging Data

In [ ]:
train.head(3)

In [ ]:
metadata.head(3)

In [ ]:
weather_train.dtypes

In [ ]:
train.columns
train_df.columns

In [ ]:
import gc
from time import time 
# %%time
train_data = pd.merge(train_df,building_meta_df,on='building_id',how='left')
# test  = pd.merge(test,metadata,on='building_id',how='left')
print ("Training Data Shape {}".format(train_data.shape))
# print ("Testing Data Shape {}".format(test.shape))
gc.collect()

In [ ]:

train_data.dtypes

In [ ]:
# %%time
train = pd.merge(train_data,weather_train,on=['site_id','timestamp'],how='left')
# test  = pd.merge(test,weather_test,on=['site_id','timestamp'],how='left')
print ("Training Data Shape {}".format(train.shape))
# print ("Testing Data Shape {}".format(test.shape))
# gc.collect()

In [ ]:
train.head(3)

In [ ]:

cols = ['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed']
for col in cols:
    train[col].fillna(train[col].mean(),inplace=True)
#     test[col].fillna(test[col].mean(),inplace=True)
    
del metadata, weather_train#, weather_test

In [ ]:
train.columns

In [ ]:
# %%time
for df in [train]:#, test]:
    df['weekend'] = np.where((df['day_of_week'] == 5) | (df['day_of_week'] == 6), 1, 0)
    df['air_temperature'] = df['air_temperature'].astype('float16')
    df['cloud_coverage'] = df['cloud_coverage'].astype("float16")
    df['dew_temperature'] = df['dew_temperature'].astype('float16')
    df['precip_depth_1_hr'] = df['precip_depth_1_hr'].astype('float32')
    df['sea_level_pressure'] = df['sea_level_pressure'].astype('float32')
    df['wind_direction'] = df['wind_direction'].astype('float32')
    df['wind_speed'] = df['wind_speed'].astype('float16')
    df['square_feet'] = df['square_feet'].astype("float32")
    df['building_id'] = df['building_id'].astype("int16")

# Supervised Learning

## Prepare the data for LSTM model

In [12]:
# load dataset
dataset = read_csv('pollution.csv', header=0, index_col=0)
values = dataset.values
# integer encode direction
encoder = LabelEncoder()
values[:,4] = encoder.fit_transform(values[:,4])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)
# drop columns we don't want to predict
reframed.drop(reframed.columns[[9,10,11,12,13,14,15]], axis=1, inplace=True)
print(reframed.head())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

the inputs (X) are reshaped into the 3D format expected by LSTMs, namely [samples, timesteps, features].

## fit model

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
# this is a layeer with 50 neurons and input_shape is 1 time searies haveing 8 features 
model.add(Dense(1)) # Dense is how many output we would like to have 
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

## Evaluate Model

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

## Complete Model

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, -7:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -7:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse) 

In [88]:
val = {'x':[1,2,3,4,5], 'y':[30,40,50,60,70], 'z':[70,80,90,100,110]}
import pandas as pd
import numpy as np
df = pd.DataFrame(val)
dataset = df[['y']].values
print(type(dataset))
dataset

<class 'numpy.ndarray'>


array([[30],
       [40],
       [50],
       [60],
       [70]])

In [93]:
len(dataset)

5

In [94]:
dataset = np.reshape(dataset, (-1,1))
type(dataset)
len(dataset)

5

In [102]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
X_train_size = len(dataset) * 0.8
X_test_size = len(dataset) - X_train
print(X_train_size, X_test_size)
dataset

4.0 1.0


array([[0.  ],
       [0.25],
       [0.5 ],
       [0.75],
       [1.  ]])

In [104]:
X_train, X_test = dataset[0:X_train_size,:], dataset[X_train_size,:]

TypeError: slice indices must be integers or None or have an __index__ method

In [77]:
data = scaler.fit_transform(dataset)
data


array([[0.        ],
       [0.57142857],
       [0.14285714],
       [0.71428571],
       [0.28571429],
       [0.85714286],
       [0.42857143],
       [1.        ]])

In [ ]:
lst = 